## Example showing the use of Xarray to read OpenDAP Netcdf files from Sentinel

In [ ]:
%matplotlib widget

import numpy as np
import xarray as xa
import matplotlib.pyplot as plt
from skimage import exposure # We are going to rescale the image

We have now imported some libraries and will load some Sentinel 2 OpenDAP NetCDF
data from https://satellittdata.no

In [ ]:
url = 'https://nbstds.met.no/thredds/dodsC/NBS/S2A/2021/07/14/S2A_MSIL1C_20210714T105031_N0301_R051_T33WWR_20210714T130146.nc#fillmismatch'
dset = xa.open_dataset(url) # opening OpenDAP NetCDF dataset


Having opened the dataset we can how have a look at the header

In [ ]:
print(dset)

Notice that it contains coordinates, variables, and attributes. We can for
instance take a closer look at the time coordinate

In [ ]:
print(dset.time)

and the B2 data variable

In [ ]:
print(dset.B2)

**What is B2?**

We can plot B2 to see it better

In [ ]:
view =dset.B2[0,100:1000,100:1000]
view.plot(cmap='Greys')
plt.show()

In [ ]:
print(dset.TCI)

In [ ]:
# East - west
xmin = 9000
xmax = -1

# North - south
ymin = 0
ymax = 2000
extent = np.asarray(
            [dset.lon[ymin,xmin].data, dset.lon[ymin,xmax].data,
            dset.lat[ymax,xmax].data, dset.lat[ymin,xmin].data]
         )

print('lon and lat',(extent)) # These are the latitudes and longitudes

# Reading in some parts of the r b g bands
b = dset.B2[0,ymin:ymax,xmin:xmax]/float(dset.attrs['QUANTIFICATION_VALUE'])
g = dset.B3[0,ymin:ymax,xmin:xmax]/float(dset.attrs['QUANTIFICATION_VALUE'])
r = dset.B4[0,ymin:ymax,xmin:xmax]/float(dset.attrs['QUANTIFICATION_VALUE'])

P0, P1 = (0, 0.15) # These are the percentages we are going to rescale on 

# Rescaling the exposures using scikit-image 
r = exposure.rescale_intensity(r,in_range=(P0,P1))
g = exposure.rescale_intensity(g,in_range=(P0,P1))
b = exposure.rescale_intensity(b,in_range=(P0,P1))

# Making the RGB image
rgb = np.dstack((r,g,b))

# Clipping it
rgb[rgb>1]=1.0


# Plotting the RGB image
plt.figure()
plt.imshow(rgb, extent = extent, interpolation='none', aspect='auto')
plt.show()